# Intro and Resource

This notebook runs the tensorflow code that trains a GRU model that creates Shakespear with given prompt.

Follows the tutorial listed on [this page](https://towardsdatascience.com/create-your-own-artificial-shakespeare-in-10-minutes-with-natural-language-processing-1fde5edc8f28).

## Installing TF and other packages

```
* TODO: create and activate new Conda Env with python 3.8.5 with following commands
    * `conda create -y -n style_gen anaconda python=3.8.5`
    * `conda activate style_gen`
* (didn't work w Python 3.9.12 LMAO) to install TensorFlow, follow step 3 and 4 in [this SO answer](https://stackoverflow.com/a/68214296/6716783).
```

* SCRATCH THIS^^^ TRY [this tutorial](https://www.mrdbourke.com/setup-apple-m1-pro-and-m1-max-for-machine-learning-and-data-science/) in your terminal
* Make env available for jupyter notebook with:
    * `conda install -y -c anaconda ipykernel`
        * `python -m ipykernel install --user --name=style_gen_env_tf292`
* In DataSpell, choose style_gen_env as the environment to use.



## Run imports

If TF installs correctly, the following cell should run with no error.

In [1]:
import tensorflow as tf    # takes a minute or two but will import eventually!
import numpy as np
import os

In [2]:
# Check for TensorFlow GPU access
print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")

# See TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.9.2


# Load Corpus

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read()
text = text.decode(encoding='utf-8')
print ('Total number of characters in the corpus is:', len(text))
print('The first 100 characters of the corpus are as follows:\n', text[:100])

Total number of characters in the corpus is: 1115394
The first 100 characters of the corpus are as follows:
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
path_to_file

'/Users/qhong/.keras/datasets/shakespeare.txt'

# Text Vectorization

In [6]:
# The unique characters in the corpus
vocab = sorted(set(text))
print ('The number of unique characters in the corpus is', len(vocab))
print('A slice of the unique characters set:\n', vocab[:10])

The number of unique characters in the corpus is 65
A slice of the unique characters set:
 ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3']


In [7]:
# Create a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
# Make a copy of the unique set elements in NumPy array format for later use in the decoding the predictions
idx2char = np.array(vocab)
# Vectorize the text with a for loop
text_as_int = np.array([char2idx[c] for c in text])

# Create Dataset

In [8]:

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
# for i in char_dataset.take(5):
#   print(i.numpy())
seq_length = 100 # The max. length for single input
# examples_per_epoch = len(text)//(seq_length+1) # double-slash for “floor” division
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
# for item in sequences.take(5):
#   print(repr(''.join(idx2char[item.numpy()])))

Metal device set to: Apple M1


2022-08-27 18:31:06.366467: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-27 18:31:06.366802: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
BUFFER_SIZE = 10000 # TF shuffles the data only within buffers

BATCH_SIZE = 64 # Batch size

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


# Build Model

In [11]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

In [12]:

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size = len(vocab), # no. of unique characters
    embedding_dim=embedding_dim, # 256
    rnn_units=rnn_units, # 1024
    batch_size=BATCH_SIZE)  # 64 for the traning

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


# Compile and Train

In [13]:
# # UGH TF is EXTREMELY SLOW without optimization...
#
# def loss(labels, logits):
#     return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
#
# # example_batch_loss  = loss(target_example_batch, example_batch_predictions)
# # print("Prediction shape: ", example_batch_predictions.shape, " (batch_size, sequence_length, vocab_size)")
# # print("scalar_loss:      ", example_batch_loss.numpy().mean())
#
# model.compile(optimizer='adam', loss=loss)
#
# # Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# # Name of the checkpoint files
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
#
# checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_prefix,
#     save_weights_only=True)
#
# EPOCHS = 30
# history = model.fit(dataset,
#                     epochs=EPOCHS,
#                     callbacks=[checkpoint_callback])

# Generate Text

In [14]:
# import tensorflow as tf
#
# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Embedding(vocab_size, embedding_dim,
#                                   batch_input_shape=[batch_size, None]),
#         tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             stateful=True,
#                             recurrent_initializer='glorot_uniform'),
#         tf.keras.layers.Dense(vocab_size)
#     ])
#     return model
#
# # Length of the vocabulary in chars
# vocab_size = 65
# # The embedding dimension
# embedding_dim = 256
# # Number of RNN units
# rnn_units = 1024
#
# checkpoint_dir = './training_checkpoints'

In [15]:
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            16640     
                                                                 
 gru_1 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_1 (Dense)             (1, None, 65)             66625     
                                                                 
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [16]:
def generate_text(model, num_generate, temperature, start_string):
    input_eval = [char2idx[s] for s in start_string] # string to numbers (vectorizing)
    input_eval = tf.expand_dims(input_eval, 0) # dimension expansion
    text_generated = [] # Empty string to store our results
    model.reset_states() # Clears the hidden states in the RNN

    for i in range(num_generate): #Run a loop for number of characters to generate
        predictions = model(input_eval) # prediction for single character
        predictions = tf.squeeze(predictions, 0) # remove the batch dimension

        # using a categorical distribution to predict the character returned by the model
        # higher temperature increases the probability of selecting a less likely character
        # lower --> more predictable
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # The predicted character as the next input to the model
        # along with the previous hidden state
        # So the model makes the next prediction based on the previous character
        input_eval = tf.expand_dims([predicted_id], 0)
        # Also devectorize the number and add to the generated text
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

generated_text = generate_text(
    model,
    num_generate=500,
    temperature=1,
    start_string=u"the work of art in the age of its technological reproducibility")
print(generated_text)

the work of art in the age of its technological reproducibility gentle
We ween Engloness for the manner,
And he pais not say we have day repose?

GRUMIO:
Nay, sir,, as Senot Eill he be man, he did's man's pawith,
I would deport you my seem to die your keep' n murder me.
Cruple master, if yet old prayers
The queen thy so lockst I hid heavy are wated with Jod,
If be incle than one comfort thangs,
Thus dares lingtweston medes my tongus, that ana
Dist for neet image hoposity heast sood.

RICHARD:
I'll know while. Let thou devise
then certoone,
And wateration an
